In [25]:
import numpy as np
import numpy.linalg as LA
import cvxpy as cp
np.random.seed(2023)

In [29]:

n = 50 # signal dimension
m = 100 # measurement dimension
alpha = 0.2 # noise parameter
noise = True
A = np.random.rand(m,n) + np.random.rand(m,n)*1j
x0 = np.random.rand(n) + np.random.rand(n)*1j
y = A@x0
b = np.abs(y) + np.random.normal(loc=0,scale=alpha)

#Demonstrate that phase retrieval is needed.
assert np.all(np.isclose(x0,np.linalg.pinv(A)@y))
assert not(np.all(np.isclose(x0,LA.pinv(A)@b)))

In [ ]:
M = np.diag(b) @ (np.eye(n)-A @ np.linalg.pinv(A)) @ np.diag(b)

In [30]:
def reconstruct(b,p):
    real = b * np.cos(p)
    imag = b * np.sin(p)
    return (real + 1j* imag)

def norm_err(xhat):
    return LA.norm(b - np.abs(A @ xhat))/ LA.norm(b)

In [42]:
## PhaseCut ##
def phasecut(b):
    X = cp.Variable((m,m),hermitian = True)
    M = np.diag(b)@ (np.eye(m) - A @ LA.pinv(A)) @ np.diag(b)
    obj = cp.norm(cp.trace(X@M))
    constr = [X >> 0]
    constr += [cp.diag(X) == np.ones(m,)]
    prob = cp.Problem(cp.Minimize(obj), constr)
    prob.solve(solver=cp.SCS)
    phase = LA.eig(X.value)[1][:,0]
    x_hat = reconstruct(b,phase)
    err = norm_err(LA.pinv(A)@x_hat)
    print("PhaseCut error: %0.4f" % err)
    return x_hat,phase, err

In [49]:
xhat,phase,err = phasecut(b)

PhaseCut error: 0.0627


array([2.00192100e-01+0.28371403j, 8.23455345e-01+0.60192756j,
       6.02365719e-01+0.16121845j, 3.13185163e-01+0.68179705j,
       9.95182171e-01+0.01711977j, 1.50947199e-01+0.1220498j ,
       3.39746531e-01+0.6859952j , 4.32910765e-01+0.57293963j,
       8.40342493e-01+0.50610145j, 9.04766161e-01+0.92398965j,
       4.54097255e-01+0.85896076j, 3.01505790e-01+0.77392789j,
       9.33181112e-01+0.48657699j, 4.65572669e-01+0.72696878j,
       6.74180607e-01+0.23368212j, 8.97752547e-01+0.28726454j,
       9.74504927e-01+0.3346534j , 2.59534482e-01+0.39625723j,
       1.68584269e-01+0.94338187j, 3.81125546e-01+0.90444729j,
       2.53563197e-02+0.04175519j, 4.87782099e-01+0.73289626j,
       1.42711102e-01+0.5611844j , 7.46940316e-01+0.26188107j,
       7.79988457e-01+0.70588441j, 5.13514885e-01+0.00825609j,
       4.85544759e-02+0.91896878j, 4.54595348e-01+0.11595447j,
       4.40627538e-02+0.53214902j, 4.83949062e-01+0.92658491j,
       6.12888988e-01+0.30176561j, 3.68651722e-01+0.035

In [51]:
b

array([23.95408785, 23.23680938, 24.84494339, 28.05465743, 26.0383445 ,
       25.92954708, 24.70398848, 25.73832157, 25.83847096, 25.92056799,
       22.25366013, 22.77007347, 25.30780578, 25.86026669, 22.01412945,
       27.00001659, 25.0614957 , 24.7422531 , 26.32496166, 22.63206947,
       26.50327535, 24.52442873, 24.35751331, 27.54874765, 27.17470332,
       22.4380928 , 26.42229759, 27.97717471, 25.21895449, 23.41707063,
       25.58854291, 22.45746759, 24.47263146, 27.92458232, 23.73160988,
       22.63856104, 23.35128847, 25.86503957, 22.27872097, 24.39139866,
       23.86771144, 27.36566979, 24.3359487 , 25.71078379, 26.07983892,
       26.52054629, 23.5274778 , 25.69716111, 26.61540892, 23.21527008,
       25.71335852, 27.00073327, 24.14187076, 25.15757258, 24.45617244,
       24.29526952, 25.70278022, 23.48841025, 28.52617608, 24.38476069,
       25.04272871, 25.96681263, 25.99712339, 25.08417283, 24.91058477,
       25.68433693, 21.66172149, 25.33112281, 28.46301384, 24.57

In [ ]:
U = U.value

B, C = np.linalg.eig(U);

u = C[:,1];
for i in 1:n
    u[i] = u[i]/abs(u[i])
end

b_angle_hat = atan.(imag.(u) ./ real.(u))
b_angle_true = atan.( imag.(b_true) ./ real.(b_true))
x_angle = atan.(imag.(x) ./ real.(x))